In [1]:
from proj1_helpers import *
import matplotlib.pyplot as plt
import numpy as np
from implementation import *
from Visualization import *
from data_preproc import preprocessing
from features_func import generate_features
from Learning import learning
from crossval import *

In [2]:
train_path = 'data/train.csv'
y, tX, ids = load_csv_data(train_path, sub_sample=False)
nSample_tr, nFeature_tr = tX.shape

test_path = 'data/test.csv'
_, tX_te, ids_te = load_csv_data(test_path, sub_sample=False)
y_te = np.zeros((tX_te.shape[0],))
nSample_te, nFeature_te = tX.shape

complete_tX = np.vstack((tX,tX_te))
complete_y = np.append(y,y_te)
complete_ids = np.append(ids,ids_te)

In [3]:
model0, model1, model2 = preprocessing(complete_tX,complete_y,complete_ids)

all_model = [model0, model1, model2]

In [4]:
print(model1['tX_te'].shape)

(175338, 18)


In [5]:
best_param = [4.9417133613238385e-08,1.0985411419875572e-06,1.4563484775012444e-06]
degree = 1

y_final = []
ids_final = []

f_scores = []

for i, model_i in enumerate(all_model):
    print(model_i['tX_tr'].shape)
    print(model_i['y_tr'].shape)
    print(model_i['tX_te'].shape)
    print(model_i['te_id'].shape)

    scores = []
    nSample, nFeature = model_i['tX_tr'].shape

    print('ridge regression')
    tX_newfeat = generate_features(model_i['tX_tr'], degree)
    w,_ = ridge_regression(model_i['y_tr'], tX_newfeat, best_param[i])
    
    tX_te_newfeat = generate_features(model_i['tX_te'], degree)
    
    pred = predict_labels(w, tX_te_newfeat)
    ids_final = np.append(ids_final, model_i['te_id'])
    y_final = np.append(y_final, pred)
    print('ids_final shape')
    print(ids_final.shape)
    print('y_final shape')
    print(y_final.shape)
    
    
    seed = 1
    k_fold = 4

    # split data in k fold
    k_indices = build_k_indices(model_i['y_tr'], k_fold, seed)
    for k in range(k_fold):
        _,_,score = cross_validation(model_i['y_tr'], tX_newfeat, k_indices, k, best_param[i])
        score *= 100
        f_scores.append(score)
        

    
    
f_scores

(99913, 16)
(99913,)
(227458, 16)
(227458,)
ridge regression
[[7.04220313 4.41176746 4.31512532 ... 1.09292951 3.4648918  4.29570593]
 [7.06975848 2.88552671 4.85736756 ... 1.45931245 1.28481519 3.70014081]
 [7.00813803 3.94268799 4.55249699 ... 1.31237868 3.65537438 4.76325028]
 ...
 [0.         4.08053295 4.12653743 ... 1.33841628 3.86342082 4.21552921]
 [6.99846489 3.01357219 4.13823385 ... 0.967364   2.56809824 4.60557011]
 [0.         4.30073523 4.1699312  ... 1.60843741 3.72858082 4.46276525]]
[[7.04220313 4.41176746 4.31512532 ... 1.09292951 3.4648918  4.29570593]
 [7.06975848 2.88552671 4.85736756 ... 1.45931245 1.28481519 3.70014081]
 [7.00813803 3.94268799 4.55249699 ... 1.31237868 3.65537438 4.76325028]
 ...
 [0.         4.08053295 4.12653743 ... 1.33841628 3.86342082 4.21552921]
 [6.99846489 3.01357219 4.13823385 ... 0.967364   2.56809824 4.60557011]
 [0.         4.30073523 4.1699312  ... 1.60843741 3.72858082 4.46276525]]
shape generated features
(99913, 60)
61
()
(0,)
(1,

shape generated features
(72543, 96)
97
()
(0,)
(1,)
(2,)
(3,)
(4,)
(5,)
(6,)
(7,)
(8,)
(9,)
(10,)
(11,)
(12,)
(13,)
(14,)
(15,)
(16,)
(17,)
(18,)
(19,)
(20,)
(21,)
(22,)
(23,)
(24,)
(25,)
(26,)
(27,)
(28,)
(29,)
(30,)
(31,)
(32,)
(33,)
(34,)
(35,)
(36,)
(37,)
(38,)
(39,)
(40,)
(41,)
(42,)
(43,)
(44,)
(45,)
(46,)
(47,)
(48,)
(49,)
(50,)
(51,)
(52,)
(53,)
(54,)
(55,)
(56,)
(57,)
(58,)
(59,)
(60,)
(61,)
(62,)
(63,)
(64,)
(65,)
(66,)
(67,)
(68,)
(69,)
(70,)
(71,)
(72,)
(73,)
(74,)
(75,)
(76,)
(77,)
(78,)
(79,)
(80,)
(81,)
(82,)
(83,)
(84,)
(85,)
(86,)
(87,)
(88,)
(89,)
(90,)
(91,)
(92,)
(93,)
(94,)
(95,)
txfinal
[[1.         0.92659051 0.76254781 ... 0.55100829 0.81406412 0.45799581]
 [1.         0.92083112 0.51511457 ... 0.68647074 0.73312643 0.72267316]
 [1.         0.92777453 0.65343487 ... 0.69355359 0.72494883 0.69205671]
 ...
 [1.         0.         0.70520445 ... 0.82667535 0.75782673 0.74238234]
 [1.         0.92561602 0.30644225 ... 0.97074529 0.76300418 0.27235176]
 [1.         

[array([82.68075907]),
 array([82.74881896]),
 array([83.33333333]),
 array([83.41740732]),
 array([75.05932116]),
 array([74.90457031]),
 array([75.7144331]),
 array([75.03868771]),
 array([76.72456576]),
 array([76.68596636]),
 array([76.71353736]),
 array([77.44141163])]

In [6]:
print(y_final.shape)

(568238,)


In [7]:
create_csv_submission(ids_final, y_final, "final_submission.csv")